<img style="float: right;" src="images/Earthwave_logo.png" alt="earthwave" width="90"/> <img style="float: right;" src="images/UZH_logo.jpg" alt="uzh" width="200"/> <img style="float: right;" src="images/ESA_logo.png" alt="esa" width="150"/>

<img src="images/GlaMBIE_logo.png" alt="logo" width="400"/>

#<strong><font color='#11847C'> Gla</font>cier <font color='#11847C'>M</font>ass <font color='#11847C'>B</font>alance <font color='#11847C'>I</font>ntercomparison <font color='#11847C'>E</font>xercise - *A community effort to reconcile measurements of glacier mass balance*</strong>

##This Jupyter Notebook can be used to explore and visualise the final results from the Glacier Mass Balance Intercomparison Exercise (GlaMBIE). In the following code, you can produce plots that demonstrate the global results from the GlaMBIE project. You can also investigate the contributions from individual regions to the global picture.

<div>

## <strong>Getting started</strong>
</div>

First, we import the necessary packages

In [1]:
from google.cloud.storage import Client
import pandas as pd
import glob, os
from pathlib import Path

from helpers import glambie_regions_dropdown, derivative_to_cumulative
from plotting import single_region_derivative_plot, single_region_cumulative_plot, region_comparison_plot

import seaborn as sns
sns.set_theme(palette='muted', style='whitegrid', font_scale=1.2)

In [2]:
# This line is only necessary when running locally. Once data is available online, it will be downloaded in 
# above cell to the local Colab working directory, so won't need to add local path to the start of 'data_directory' below
local_path = '/data/ox1/working/glambie/demo_notebooks'

In [4]:
temporal_option = 'calendar_years'
data_directory = os.path.join(local_path, 'glambie_csvs_to_publish_v3', temporal_option)

In [5]:
global_dataframe = pd.read_csv(os.path.join(data_directory, '0_global.csv'))